<a href="https://colab.research.google.com/github/aborundiya/CNN/blob/master/CIFAR100_SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import copy 
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim 
from torchvision import models
import matplotlib.pyplot as plt 

#Check for GPU 

In [79]:
device = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [0]:
def transform_download_load(imgSize=224 , norm_mean= (0.5,0.5,0.5), norm_std = (0.5,0.5,0.5),dataset=torchvision.datasets.CIFAR10,batch_size=16): 

  transform_train = transforms.Compose([
                                      transforms.RandomResizedCrop(299),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
  ])

  transform_test = transforms.Compose([
                                     transforms.RandomResizedCrop(299),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
  ])
  train_set = dataset('./data',train=True,download=True,transform=transform_train)
  test_set = dataset('./data',train=False,download=True,transform=transform_test)
  trainloader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
  testloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False)

  return trainloader, testloader

In [0]:
def model_initialize(model, pretain=True):
   net = model(pretrained=True)
   for param in net.parameters():
     param.requires_grad = False
   return net

In [0]:
def evaluation(dataloader, model):
  total , correct = 0 , 0 
  for data in dataloader: 
    inputs, labels = data 
    inputs, labels = inputs.to(device) , labels.to(device)
    outputs  = model(inputs)
    _ , pred = torch.max(outputs.data , 1 )
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  return 100 * (correct / total )

In [0]:


def fit(function,lr,weight_decay,loss_fn,net,train_loader,test_loader,batch_size):
  for i in range(len(lr)):
    for j in range(len(weight_decay)):
      for k in range(len(function)):
        opt = function[k](net.parameters(),lr=lr[i], weight_decay=weight_decay[j])
        loss_arr = []
        loss_arr_epoch = []
        max_epoch = 1 
        n_iter = np.ceil(50000/batch_size)
        print(function[k])
        print("For weight decay : %f and For learning Rate : %f:" % (weight_decay[j], lr[i]))
        min_loss = 1000

        for epoch in range(max_epoch):
          for l , data in enumerate(train_loader,0):
            inputs, labels = data 
            inputs, labels = inputs.to(device), labels.to(device)
            opt.zero_grad()
            outputs= net(inputs)
            loss = loss_fn(outputs,labels)
            loss.backward()
            opt.step()
            if min_loss > loss.item():
              min_loss = loss.item()
              best_model = copy.deepcopy(net.state_dict())
              #print("Min Loss %0.2f"% min_loss)
            #if l % 100 == 0 : 
              #print("Epoch %d/%d, Loss : %0.2f"  % (l, n_iter,loss.item()))
            
            del inputs, labels, outputs 
            torch.cuda.empty_cache()
          loss_arr_epoch.append(loss.item())
          net.load_state_dict(best_model)
          print("Epoch %d/%d , Test Accuracy : %0.2f , Train Accuracy : %0.2f" % (epoch, max_epoch, evaluation(test_loader,net), evaluation(train_loader,net)))


In [84]:
batch_size = 128 
num_classes = 100
trainloader , testloader = transform_download_load(imgSize=299, batch_size=batch_size,dataset=torchvision.datasets.CIFAR100)



Files already downloaded and verified
Files already downloaded and verified


In [0]:
 
squeezenet = models.squeezenet1_1(pretrained=True)
for param in squeezenet.parameters():
  param.requires_grad = False

In [86]:
print(squeezenet)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

# Altering the aux and Fc layer to suit the CIFAR dataset

In [87]:

final_in_channels = squeezenet.classifier[1].in_channels
squeezenet.classifier[1] = nn.Conv2d(final_in_channels,num_classes,kernel_size=(1,1),stride=(1,1))
print(squeezenet.classifier)


Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Conv2d(512, 100, kernel_size=(1, 1), stride=(1, 1))
  (2): ReLU(inplace=True)
  (3): AdaptiveAvgPool2d(output_size=(1, 1))
)


In [88]:
for param in squeezenet.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([100, 512, 1, 1])
torch.Size([100])


#Training and evaluation of the network

In [0]:

squeezenet = squeezenet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(squeezenet.parameters(), lr = 0.01)
loss_fn = nn.CrossEntropyLoss()
lr = [0.01]
#lr = [0.1,0.01,0.001,0.0001]
weight_decay = [0.0001]
function = [optim.SGD, optim.Adam, optim.Adagrad, optim.RMSprop]
fit(function=function,lr=lr,weight_decay=weight_decay,loss_fn=loss_fn,net=squeezenet,train_loader=trainloader,test_loader=testloader,batch_size=batch_size)


In [0]:

print("Epoch %d/%d , Test Accuracy : %0.2f , Train Accuracy : %0.2f" % (0, 1, evaluation(testloader,squeezenet), evaluation(trainloader,squeezenet)))